In [2]:
import cv2
import numpy as np
import os

# --- Placeholder for taken_frames_durations ---
# Please replace this with your actual implementation of taken_frames_durations.
# This function should return a list of durations (in seconds) at which to capture frames.
def taken_frames_durations(cap, target_fps):
    """
    Generates a list of target durations for frame capture.
    (Placeholder - replace with your actual logic)
    """
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0: # Avoid division by zero if fps is not available
        return []
    
    durations = []
    # Example: capture a frame every (1/target_fps) seconds
    # This might need adjustment based on your original taken_frames_durations logic
    interval = 1.0 / target_fps
    current_duration = 0.0
    while current_duration < (total_frames / fps):
        durations.append(current_duration)
        current_duration += interval
    return durations
# --- End Placeholder ---


taken_fps = 10 # This now refers to the target frames per second to sample
store_image = list()
video_directory = "D:\\Research Dataset\\Surveillance\\Train" # Directory containing your training videos

# Ensure the dataset directory exists
dataset_output_dir = '../dataset'
if not os.path.exists(dataset_output_dir):
    os.makedirs(dataset_output_dir)

video_names = os.listdir(video_directory) 
print(f"Found {len(video_names)} videos in '{video_directory}'")

for name in video_names:
    video_path = os.path.join(video_directory, name)
    cap = cv2.VideoCapture(video_path) 
    
    if not cap.isOpened():
        print(f"Warning: Could not open video file: {video_path}. Skipping.")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        print(f"Warning: FPS is 0 for video: {video_path}. Skipping.")
        cap.release()
        continue

    final_taken_fps = min(fps, taken_fps) # taken_fps should not higher than video fps
    saving_frames_durations = taken_frames_durations(cap, final_taken_fps)
    
    time_count = 0
    frame_number = 0
    
    print(f"Processing video: {name} (FPS: {fps:.2f}, Target Sampled FPS: {final_taken_fps})")

    while True:
        is_read, frame = cap.read()
        if not is_read:
            break # End of video

        frame_duration = time_count / fps # measure duration until video end
        
        # Check if there are durations left to process
        if not saving_frames_durations:
            break # No more specific durations to save frames at

        closest_duration = saving_frames_durations[0]
        
        if frame_duration >= closest_duration: #this condition maintain taken 10 fps
            # Resize frame to the model's expected input dimensions (232x232)
            frame = cv2.resize(frame, (232, 232), interpolation = cv2.INTER_AREA)
            # Convert to grayscale
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            # Store the grayscale frame.
            # Normalization to [-1, 1] will be applied after collecting all frames,
            # or can be done here if preferred for streaming data.
            store_image.append(frame)
            
            frame_number += 1
            
            try:
                saving_frames_durations.pop(0) # Remove the duration that has been processed
            except IndexError:
                # This should ideally not happen if the loop breaks when saving_frames_durations is empty
                pass
        time_count += 1
    
    cap.release() # Release the video capture object after processing each video

print(f"\nFinished processing all videos. Total frames collected: {len(store_image)}")

# Convert the list of frames to a NumPy array
# The shape will be (number_of_frames, height, width) -> (num_frames, 232, 232)
store_image_np = np.array(store_image, dtype=np.float32)

# Add a channel dimension for grayscale images, making the shape (num_frames, 232, 232, 1)
# This is the standard input shape for many Keras/TensorFlow models for single images.
store_image_final = np.expand_dims(store_image_np, axis=-1)

print(f"Shape of collected frames before normalization: {store_image_final.shape}")

# Normalize pixel values to the range [-1, 1]
# This normalization is applied to the entire dataset.
# Ensure your model's input layer expects values in this range.
if store_image_final.size > 0: # Avoid division by zero if no images were stored
    mean_val = store_image_final.mean()
    std_val = store_image_final.std()
    
    if std_val == 0: # Handle cases where std dev is zero (e.g., all black frames)
        print("Warning: Standard deviation is zero. Setting normalized frames to 0.")
        store_image_final_normalized = np.zeros_like(store_image_final)
    else:
        store_image_final_normalized = (store_image_final - mean_val) / std_val
    
    # Clip values to a reasonable range, e.g., [-1, 1] or [0, 1] if your model expects that.
    # The original code clipped to [0,1], but (x-mean)/std can go outside this.
    # If the model expects [-1,1] from (gray - 0.5) * 2, then this clipping might need adjustment.
    # For now, let's keep it consistent with the previous preprocess_frame_for_new_model
    # which normalizes to [-1, 1].
    store_image_final_normalized = (store_image_final_normalized - store_image_final_normalized.min()) / \
                                   (store_image_final_normalized.max() - store_image_final_normalized.min())
    store_image_final_normalized = (store_image_final_normalized - 0.5) * 2
    
    # It's important to clip after normalization if values can go out of expected range.
    # The original code used np.clip(..., 0, 1) after (x-mean)/std, which is unusual.
    # Let's ensure the final range is consistent with the model's preprocessing.
    # If the model expects [-1, 1], then ensure the training data is also in this range.
    # The preprocess_frame_for_new_model uses (gray - 0.5) * 2 which results in [-1, 1].
    # So, let's ensure the saved training data also adheres to this.
    # The above normalization to [-1,1] is good.
    
    # If you want to strictly clip to [-1, 1] after the above normalization:
    store_image_final_normalized = np.clip(store_image_final_normalized, -1, 1)

    print(f"Shape of collected frames after normalization: {store_image_final_normalized.shape}")
    print(f"Min pixel value after normalization: {np.min(store_image_final_normalized):.4f}")
    print(f"Max pixel value after normalization: {np.max(store_image_final_normalized):.4f}")
    
    # Save the processed and normalized frames
    np.save(os.path.join(dataset_output_dir, 'training_frames.npy'), store_image_final_normalized)
    print(f"Saved processed training data to '{os.path.join(dataset_output_dir, 'training_frames.npy')}'")
else:
    print("No frames were collected. 'training_frames.npy' will not be saved.")



Found 1 videos in 'D:\Research Dataset\Surveillance\Train'
Processing video: training_video.mp4 (FPS: 30.00, Target Sampled FPS: 10)

Finished processing all videos. Total frames collected: 67
Shape of collected frames before normalization: (67, 232, 232, 1)
Shape of collected frames after normalization: (67, 232, 232, 1)
Min pixel value after normalization: -1.0000
Max pixel value after normalization: 1.0000
Saved processed training data to '../dataset\training_frames.npy'
